# Day8_3: 시계열 정상성과 진단 - 정답

이 노트북은 Day8_3 실습 퀴즈의 정답과 상세 풀이를 포함합니다.

---

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 로드 완료")

In [ ]:
# 공통 함수 정의

def adf_test(series, name=''):
    """ADF 검정 수행"""
    result = adfuller(series.dropna())
    print(f'>>> ADF 검정: {name} <<<')
    print(f'- ADF 통계량: {result[0]:.4f}')
    print(f'- p-value: {result[1]:.4f}')
    if result[1] < 0.05:
        print("✅ 결론: 정상 시계열")
        return True
    else:
        print("❌ 결론: 비정상 시계열")
        return False

def kpss_test(series, name=''):
    """KPSS 검정 수행"""
    result = kpss(series.dropna(), regression='c', nlags='auto')
    print(f'>>> KPSS 검정: {name} <<<')
    print(f'- KPSS 통계량: {result[0]:.4f}')
    print(f'- p-value: {result[1]:.4f}')
    if result[1] > 0.05:
        print("✅ 결론: 정상 시계열")
        return True
    else:
        print("❌ 결론: 비정상 시계열")
        return False

def check_stationarity(series, name=''):
    """ADF + KPSS 종합 판단"""
    adf_result = adfuller(series.dropna())
    kpss_result = kpss(series.dropna(), regression='c', nlags='auto')
    
    adf_stationary = adf_result[1] < 0.05
    kpss_stationary = kpss_result[1] > 0.05
    
    print(f"\n📊 종합 검정: {name}")
    print(f"ADF:  p={adf_result[1]:.4f} → {'✅정상' if adf_stationary else '❌비정상'}")
    print(f"KPSS: p={kpss_result[1]:.4f} → {'✅정상' if kpss_stationary else '❌비정상'}")
    
    if adf_stationary and kpss_stationary:
        print("🎯 종합: 정상 시계열")
        return 'stationary'
    elif not adf_stationary and not kpss_stationary:
        print("🎯 종합: 비정상 시계열")
        return 'non-stationary'
    else:
        print("🎯 종합: 추가 분석 필요")
        return 'uncertain'

---

## Q1. 정상 vs 비정상 구분하기 ⭐

**문제**: 다음 두 시계열 중 어느 것이 정상 시계열인지 시각화하여 판단하세요.

In [ ]:
# 정답 코드

# 데이터 생성
np.random.seed(100)
series_a = pd.Series(np.random.randn(100))          # 백색 소음 (정상)
series_b = pd.Series(np.cumsum(np.random.randn(100)))  # 랜덤 워크 (비정상)

# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=('Series A', 'Series B'))

fig.add_trace(
    go.Scatter(y=series_a, mode='lines', name='A', line=dict(color='blue')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=series_b, mode='lines', name='B', line=dict(color='red')),
    row=1, col=2
)

fig.update_layout(height=400, title_text='Series A vs Series B')
fig.show()

# 판단
print("\n📊 판단 결과:")
print("Series A: 평균이 0 근처에서 일정하게 변동 → 정상 시계열")
print("Series B: 시간에 따라 평균이 변함 (추세 존재) → 비정상 시계열")

In [ ]:
# 검증: ADF 검정으로 확인
print("=" * 50)
adf_test(series_a, 'Series A')
print()
adf_test(series_b, 'Series B')

### 💡 풀이 설명

**접근 방법**:
1. 시각적으로 데이터의 패턴 확인
2. 평균이 일정한지, 추세가 있는지 판단

**핵심 개념**:
- `np.random.randn()`: 표준정규분포에서 추출 → 평균 0, 분산 1로 일정
- `np.cumsum()`: 누적합 → 랜덤워크 생성, 평균이 시간에 따라 변동

**실무 팁**:
- 시각화 먼저! 눈으로 추세/계절성 확인 후 통계 검정

---

## Q2. ADF 검정 수행하기 ⭐⭐

**문제**: 다음 시계열 데이터에 대해 ADF 검정을 수행하고 정상성을 판단하세요.

In [ ]:
# 정답 코드

# 데이터 생성 (선형 추세 + 노이즈)
np.random.seed(42)
test_series = pd.Series(np.linspace(0, 10, 100) + np.random.randn(100) * 2)

# 시각화
fig = px.line(test_series, title='Test Series (선형 추세 + 노이즈)')
fig.show()

# ADF 검정
result = adfuller(test_series)

print(">>> ADF 검정 결과 <<<")
print(f"ADF 통계량: {result[0]:.4f}")
print(f"p-value: {result[1]:.4f}")
print(f"사용된 시차: {result[2]}")
print(f"관측치 수: {result[3]}")
print("\n임계값:")
for key, value in result[4].items():
    print(f"  {key}: {value:.4f}")

print("\n📊 결론:")
if result[1] < 0.05:
    print("p-value < 0.05 → 정상 시계열")
else:
    print("p-value >= 0.05 → 비정상 시계열 (선형 추세 존재)")

### 💡 풀이 설명

**핵심 개념**:
- ADF 검정의 귀무가설: "비정상 시계열이다"
- p-value < 0.05 → 귀무가설 기각 → 정상
- p-value >= 0.05 → 귀무가설 채택 → 비정상

**주의사항**:
- `adfuller()` 반환값: (통계량, p-value, 시차, 관측치, 임계값, ...)
- `dropna()`로 결측치 제거 필수

---

## Q3. KPSS 검정 수행하기 ⭐⭐

**문제**: Q2의 동일한 데이터에 대해 KPSS 검정을 수행하세요.

In [ ]:
# 정답 코드

# KPSS 검정
result = kpss(test_series, regression='c', nlags='auto')

print(">>> KPSS 검정 결과 <<<")
print(f"KPSS 통계량: {result[0]:.4f}")
print(f"p-value: {result[1]:.4f}")
print(f"사용된 시차: {result[2]}")
print("\n임계값:")
for key, value in result[3].items():
    print(f"  {key}: {value:.4f}")

print("\n📊 결론:")
if result[1] > 0.05:
    print("p-value > 0.05 → 정상 시계열")
else:
    print("p-value <= 0.05 → 비정상 시계열")

print("\n📊 ADF vs KPSS 비교:")
print("- ADF: 비정상 (추세 존재)")
print("- KPSS: 비정상 (추세 존재)")
print("→ 두 검정 모두 비정상으로 판단")

### 💡 풀이 설명

**핵심 개념**:
- KPSS 귀무가설: "정상 시계열이다" (ADF와 반대!)
- p-value > 0.05 → 귀무가설 채택 → 정상
- p-value <= 0.05 → 귀무가설 기각 → 비정상

**파라미터 설명**:
- `regression='c'`: 상수항에 대한 정상성 검정
- `regression='ct'`: 추세+상수항에 대한 검정
- `nlags='auto'`: 자동 시차 선택

---

## Q4. 1차 차분으로 정상화하기 ⭐⭐⭐

**문제**: Q2의 데이터에 1차 차분을 적용한 후 ADF 검정으로 정상성을 확인하세요.

In [ ]:
# 정답 코드

# 1차 차분 적용
test_diff = test_series.diff().dropna()

# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=('원본', '1차 차분'))

fig.add_trace(
    go.Scatter(y=test_series, mode='lines', name='원본', line=dict(color='blue')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=test_diff, mode='lines', name='차분', line=dict(color='red')),
    row=1, col=2
)

fig.update_layout(height=400, title_text='원본 vs 1차 차분')
fig.show()

# ADF 검정
print(">>> 차분 전 ADF 검정 <<<")
adf_test(test_series, '원본')

print("\n>>> 차분 후 ADF 검정 <<<")
result = adf_test(test_diff, '1차 차분')

In [ ]:
# 검증: 차분 후 p-value < 0.05 확인
adf_result = adfuller(test_diff)
assert adf_result[1] < 0.05, "차분 후에도 비정상!"
print("✅ 검증 통과: 차분 후 정상 시계열")

### 💡 풀이 설명

**접근 방법**:
1. `.diff()`로 1차 차분 적용
2. `.dropna()`로 첫 번째 NaN 제거
3. ADF 검정으로 정상성 확인

**핵심 개념**:
- 차분: $Y'_t = Y_t - Y_{t-1}$
- 추세가 있는 데이터 → 차분 → 추세 제거

**주의사항**:
- 차분 후 데이터 개수가 1개 줄어듬

---

## Q5. 로그 변환 적용하기 ⭐⭐⭐

**문제**: 다음 지수 성장 데이터에 로그 변환을 적용하고, 변환 전후의 분산을 비교하세요.

In [ ]:
# 정답 코드

# 데이터 생성 (지수 성장)
np.random.seed(50)
exp_data = pd.Series(100 * np.exp(np.linspace(0, 2, 100)) + np.random.randn(100) * 10)

# 로그 변환
log_data = np.log(exp_data)

# 분산 비교
print(">>> 분산 비교 <<<")
print(f"원본 분산: {exp_data.var():.2f}")
print(f"로그 변환 후 분산: {log_data.var():.4f}")
print(f"분산 감소율: {(1 - log_data.var()/exp_data.var())*100:.2f}%")

# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=('원본 (지수 성장)', '로그 변환'))

fig.add_trace(
    go.Scatter(y=exp_data, mode='lines', name='원본', line=dict(color='blue')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=log_data, mode='lines', name='로그', line=dict(color='green')),
    row=1, col=2
)

fig.update_layout(height=400, title_text='로그 변환: 분산 안정화')
fig.show()

### 💡 풀이 설명

**핵심 개념**:
- 로그 변환: $Y' = \log(Y)$
- 지수 성장 데이터의 분산을 안정화
- 주가, GDP 등 기하급수적으로 성장하는 데이터에 효과적

**실무 팁**:
- 로그 변환은 분산만 안정화, 추세는 여전히 존재
- 로그 변환 후 차분 → 로그 수익률

---

## Q6. ACF 플롯 해석하기 ⭐⭐⭐

**문제**: 다음 두 시계열의 ACF 플롯을 그리고, 어느 것이 정상 시계열인지 ACF 패턴으로 판단하세요.

In [ ]:
# 정답 코드

# 데이터 생성
np.random.seed(123)
stationary = pd.Series(np.random.randn(200))
non_stationary = pd.Series(np.cumsum(np.random.randn(200)))

# ACF 플롯
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

plot_acf(stationary, lags=30, ax=axes[0])
axes[0].set_title('ACF: 정상 시계열 (백색 소음)')

plot_acf(non_stationary, lags=30, ax=axes[1])
axes[1].set_title('ACF: 비정상 시계열 (랜덤 워크)')

plt.tight_layout()
plt.show()

print("\n📊 해석:")
print("- 왼쪽 (정상): ACF가 lag 1 이후 빠르게 0에 수렴")
print("- 오른쪽 (비정상): ACF가 천천히 감소, 높은 자기상관 지속")

### 💡 풀이 설명

**핵심 개념**:
- 정상 시계열: ACF가 빠르게 0으로 수렴
- 비정상 시계열: ACF가 천천히 감소

**ACF 패턴 해석**:
- 파란 영역: 95% 신뢰구간
- 영역 밖의 값: 통계적으로 유의한 자기상관

---

## Q7. ADF + KPSS 종합 판단하기 ⭐⭐⭐⭐

**문제**: yfinance로 삼성전자(005930.KS) 주가를 가져와 ADF와 KPSS 검정을 모두 수행하고, 종합 결론을 내리세요.

In [ ]:
# 정답 코드
import yfinance as yf

# 삼성전자 주가 가져오기
samsung = yf.Ticker('005930.KS')
samsung_data = samsung.history(period='2y')
samsung_price = samsung_data['Close']

print(f"삼성전자 주가 데이터: {len(samsung_price)}일")
print(samsung_price.tail())

# 시각화
fig = px.line(samsung_price, title='삼성전자 종가 추이')
fig.show()

In [ ]:
# ADF + KPSS 검정
print("=" * 50)
adf_test(samsung_price, '삼성전자')
print()
kpss_test(samsung_price, '삼성전자')
print()

# 종합 판단
check_stationarity(samsung_price, '삼성전자 주가')

### 💡 풀이 설명

**핵심 개념**:
- 주가는 대표적인 비정상 시계열
- ADF와 KPSS 모두 비정상으로 판단

**실무 팁**:
- 두 검정 결과가 일치하면 확신 있게 판단
- 결과가 다르면 보수적으로 비정상으로 간주

---

## Q8. 로그 변환 + 차분 파이프라인 ⭐⭐⭐⭐

**문제**: Q7의 삼성전자 주가에 로그 변환 후 1차 차분을 적용하고, 각 단계별 정상성을 확인하세요.

In [ ]:
# 정답 코드

# 단계별 변환
samsung_original = samsung_price.copy()
samsung_log = np.log(samsung_price)
samsung_log_diff = samsung_log.diff().dropna()

# 각 단계별 ADF 검정
print(">>> 단계별 정상성 검정 <<<\n")

print("[1단계] 원본")
adf_test(samsung_original, '원본')

print("\n[2단계] 로그 변환")
adf_test(samsung_log, '로그')

print("\n[3단계] 로그 + 1차 차분")
adf_test(samsung_log_diff, '로그+차분')

In [ ]:
# 시각화
fig = make_subplots(rows=1, cols=3, subplot_titles=('원본', '로그 변환', '로그 + 1차 차분'))

fig.add_trace(go.Scatter(y=samsung_original, mode='lines', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(y=samsung_log, mode='lines', line=dict(color='green')), row=1, col=2)
fig.add_trace(go.Scatter(y=samsung_log_diff, mode='lines', line=dict(color='red')), row=1, col=3)

fig.update_layout(height=400, title_text='삼성전자 주가 정상화 과정', showlegend=False)
fig.show()

### 💡 풀이 설명

**변환 순서**:
1. 원본: 비정상 (추세 + 변동성 변화)
2. 로그: 여전히 비정상 (분산은 안정화되나 추세 존재)
3. 로그+차분: 정상 (추세 제거됨)

**핵심 개념**:
- 로그 차분 = 로그 수익률
- 금융 데이터 분석의 표준 방법

---

## Q9. ACF/PACF로 모델 파라미터 추정 ⭐⭐⭐⭐⭐

**문제**: Q8에서 정상화된 데이터의 ACF/PACF 플롯을 그리고, ARIMA 모델의 p, q 값을 추정하세요.

In [ ]:
# 정답 코드

# ACF/PACF 플롯
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

plot_acf(samsung_log_diff.dropna(), lags=30, ax=axes[0])
axes[0].set_title('ACF: 삼성전자 로그 수익률')

plot_pacf(samsung_log_diff.dropna(), lags=30, ax=axes[1], method='ywm')
axes[1].set_title('PACF: 삼성전자 로그 수익률')

plt.tight_layout()
plt.show()

print("\n📊 ARIMA 파라미터 추정:")
print("- ACF: lag 1에서 유의미한 값 후 절단 → q=1 추정")
print("- PACF: lag 1에서 유의미한 값 후 절단 → p=1 추정")
print("- d=1 (1차 차분 적용)")
print("\n🎯 추천 모델: ARIMA(1, 1, 1)")

### 💡 풀이 설명

**ACF/PACF 해석 규칙**:

| 패턴 | ACF | PACF | 모델 |
|------|-----|------|----- |
| 절단 | lag q 이후 | 점차 감소 | MA(q) |
| 절단 | 점차 감소 | lag p 이후 | AR(p) |
| 절단 | lag q 이후 | lag p 이후 | ARMA(p,q) |

**실무 팁**:
- 실제로는 `auto_arima`로 자동 최적화 많이 사용
- ACF/PACF는 초기 추정에 참고

---

## Q10. 완전한 정상화 파이프라인 구축 ⭐⭐⭐⭐⭐

**문제**: 다음 요구사항을 만족하는 정상화 파이프라인 함수를 작성하세요.

In [ ]:
# 정답 코드

def full_stationarize_pipeline(series, name=''):
    """
    완전한 정상화 파이프라인
    
    Returns:
    --------
    stationary_series : pd.Series
        정상화된 시계열
    transform_info : dict
        적용된 변환 정보
    """
    results = {}
    transform_info = {'log': False, 'diff': 0}
    
    print(f"\n{'='*60}")
    print(f"📊 정상화 파이프라인: {name}")
    print(f"{'='*60}")
    
    # 원본 저장
    current = series.copy()
    results['original'] = current
    
    # 1단계: 원본 검정
    print("\n[1단계] 원본 데이터")
    adf_p = adfuller(current.dropna())[1]
    print(f"ADF p-value: {adf_p:.4f}")
    
    if adf_p < 0.05:
        print("✅ 이미 정상 시계열!")
        return current, transform_info, results
    
    # 2단계: 로그 변환
    print("\n[2단계] 로그 변환")
    if (current > 0).all():  # 양수 데이터만 로그 가능
        current = np.log(current)
        results['log'] = current
        transform_info['log'] = True
        
        adf_p = adfuller(current.dropna())[1]
        print(f"ADF p-value: {adf_p:.4f}")
        
        if adf_p < 0.05:
            print("✅ 로그 변환만으로 정상화!")
            return current, transform_info, results
    else:
        print("⚠️ 음수 값 존재, 로그 변환 스킵")
    
    # 3단계: 1차 차분
    print("\n[3단계] 1차 차분")
    current = current.diff().dropna()
    results['diff1'] = current
    transform_info['diff'] = 1
    
    adf_p = adfuller(current.dropna())[1]
    print(f"ADF p-value: {adf_p:.4f}")
    
    if adf_p < 0.05:
        print("✅ 1차 차분으로 정상화!")
        return current, transform_info, results
    
    # 4단계: 2차 차분
    print("\n[4단계] 2차 차분")
    current = current.diff().dropna()
    results['diff2'] = current
    transform_info['diff'] = 2
    
    adf_p = adfuller(current.dropna())[1]
    print(f"ADF p-value: {adf_p:.4f}")
    
    if adf_p < 0.05:
        print("✅ 2차 차분으로 정상화!")
    else:
        print("⚠️ 2차 차분 후에도 비정상, 추가 분석 필요")
    
    return current, transform_info, results

In [ ]:
# 테스트 데이터
np.random.seed(777)
test_data = pd.Series(100 * np.exp(np.linspace(0, 1, 200)) + np.random.randn(200) * 5)

# 파이프라인 실행
stationary, info, results = full_stationarize_pipeline(test_data, '테스트 데이터')

print(f"\n📌 적용된 변환: 로그={info['log']}, 차분={info['diff']}차")

In [ ]:
# 시각화
n_plots = len(results)
fig = make_subplots(rows=1, cols=n_plots, subplot_titles=list(results.keys()))

for i, (name, data) in enumerate(results.items(), 1):
    fig.add_trace(
        go.Scatter(y=data, mode='lines', name=name),
        row=1, col=i
    )

fig.update_layout(height=400, title_text='단계별 변환 결과', showlegend=False)
fig.show()

In [ ]:
# ACF/PACF 플롯
print("\n📊 정상화된 데이터의 ACF/PACF:")

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

plot_acf(stationary.dropna(), lags=30, ax=axes[0])
axes[0].set_title('ACF')

plot_pacf(stationary.dropna(), lags=30, ax=axes[1], method='ywm')
axes[1].set_title('PACF')

plt.tight_layout()
plt.show()

In [ ]:
# 검증
adf_result = adfuller(stationary.dropna())
assert adf_result[1] < 0.05, "정상화 실패!"
print("\n✅ 검증 통과: 정상화 성공!")

### 💡 풀이 설명

**파이프라인 구조**:
1. 원본 검정 → 정상이면 반환
2. 로그 변환 (양수 데이터만) → 정상이면 반환
3. 1차 차분 → 정상이면 반환
4. 2차 차분 → 최종 반환

**실무 팁**:
- 차분은 보통 2차까지, 그 이상은 과적합 위험
- 로그 변환은 양수 데이터만 가능
- 변환 정보를 저장해두면 나중에 역변환 가능

---

## 📚 전체 요약

| 퀴즈 | 난이도 | 핵심 개념 |
|------|--------|----------|
| Q1 | ⭐ | 시각적 정상성 판단 |
| Q2 | ⭐⭐ | ADF 검정 수행 |
| Q3 | ⭐⭐ | KPSS 검정 수행 |
| Q4 | ⭐⭐⭐ | 차분으로 정상화 |
| Q5 | ⭐⭐⭐ | 로그 변환 |
| Q6 | ⭐⭐⭐ | ACF 플롯 해석 |
| Q7 | ⭐⭐⭐⭐ | ADF+KPSS 종합 판단 |
| Q8 | ⭐⭐⭐⭐ | 로그+차분 파이프라인 |
| Q9 | ⭐⭐⭐⭐⭐ | ACF/PACF로 p,q 추정 |
| Q10 | ⭐⭐⭐⭐⭐ | 완전한 파이프라인 구축 |